<a href="https://colab.research.google.com/github/melataguia/regretion_cluster../blob/main/reseau_neuronal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation de la bibliothèque Keras

In [ ]:
if (!require(keras)) {
  install.packages("keras")
}
if (!require(tensorflow)) {
  install.packages("tensorflow")
}


##Bibliothèques nécessaires


In [26]:
library(keras)
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




## Les données MovieLens(ratings et movies)

In [34]:
movies <- read.csv('../movies.csv')
ratings <- read.csv('../ratings.csv')
#tags <- read.csv('./tags.csv')

# Vérification des données manquantes

In [17]:
missing_valuesr <- sapply(ratings, function(x) sum(is.na(x)))
missing_valuesm <- sapply(movies, function(x) sum(is.na(x)))
print(missing_valuesr)
print(missing_valuesm)

   userId   movieId    rating timestamp 
        0         0         0         2 
movieId   title  genres 
      0       0       0 


##Supprimer les lignes avec des valeurs manquantes sur "ratings"

In [18]:
ratings <- ratings[complete.cases(ratings), ]

##Supprimer les lignes avec des valeurs manquantes sur "movies"

In [19]:
movies <- ratings[complete.cases(movies), ]

###Fusionner les données

In [35]:
merged_df <- merge(ratings, movies, by = "movieId", all.x=TRUE)

##Remplacement des valeurs manquantes par une chaîne vide

In [49]:
str(merged_df)

'data.frame':	158589 obs. of  6 variables:
 $ movieId  : num  1 1 1 1 1 1 1 1 1 1 ...
 $ userId   : int  1 232 562 432 550 179 112 471 328 468 ...
 $ rating   : num  4 3.5 4.5 2.5 4 4 3 5 5 4 ...
 $ timestamp: int  964982703 1076955621 1368893997 1316391457 1488728441 852114051 1442535639 1496671820 1494210665 831400444 ...
 $ title    : Factor w/ 9719 levels "Toy Story (1995)",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ genres   : Factor w/ 951 levels "Adventure|Animation|Children|Comedy|Fantasy",..: 1 1 1 1 1 1 1 1 1 1 ...


##Prétraitement des données
###Encoder les variables catégorielles

In [50]:
merged_df$title <- as.character(merged_df$title)
merged_df$genres <- as.character(merged_df$genres)


###Contrôler et assigner les niveaux des facteurs

In [51]:
merged_df$title <- factor(merged_df$title, levels = unique(merged_df$title))
merged_df$genres <- factor(merged_df$genres, levels = unique(merged_df$genres))


##Division des données en ensembles d'entraînement et de test

In [52]:
set.seed(42) # Pour la reproductibilité
train_indices <- sample(1:nrow(merged_df), 0.8*nrow(merged_df))
train_data <- merged_df[train_indices, ]
test_data <- merged_df[-train_indices, ]


##Création du modèle

In [ ]:

model <- keras_model_sequential()

# Ajouter des couches au modèle
model %>%
  layer_embedding(input_dim = nlevels(merged_df$title) + nlevels(merged_df$genres) + 1, output_dim = 32) %>%
  layer_flatten() %>%
  layer_dense(units = 4, activation = "relu") %>%
  layer_dense(units = 32, activation = "relu") %>%
  layer_dense(units = 1, activation = "linear") %>%
  layer_dense(units = 32, activation = "relu") %>%
  layer_dense(units = 64, activation = "relu")


# Compiler le modèle

In [46]:
model %>% compile(
  loss = "mean_squared_error",
  optimizer = optimizer_adam(),
  metrics = list("mean_absolute_error")
)

##Entraîner le modèle

In [ ]:
history <- model %>% fit(
  x = list(train_data$title, train_data$genres),
  y = train_data$rating,
  epochs = 10,
  batch_size = 64,
  validation_split = 0.2
)

##Évaluation du modèle

In [ ]:
model %>% evaluate(
  x = list(test_data$title, test_data$genres),
  y = test_data$rating
)